In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch import cuda
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torch.nn import Conv2d, ReLU, Flatten, MaxPool2d, BatchNorm2d, Dropout2d, Linear, Sequential, Module
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import math
import random
import numpy
from tabulate import tabulate

In [ ]:
from network import *
from metrics import *

In [ ]:
root = '/content/drive/MyDrive/buffer64'
batch = 64
my_transforms = transforms.Compose([transforms.Grayscale(), transforms.ToTensor()])
dataset = datasets.ImageFolder(root = root, transform = my_transforms)
size = len(dataset)
train_size = 80 * size // 100
val_size = 10 * size // 100
test_size = size - train_size - val_size
split = [train_size, val_size, test_size]
trainset, valset, testset = random_split(dataset, split)
trainloader = DataLoader(trainset, batch, shuffle = True)
valloader = DataLoader(valset, batch)
testloader = DataLoader(testset, batch)

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'using {device} device')

using cuda device


In [ ]:
net = Net().to(device)
optimizer = torch.optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 0.001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 60
step = 10
trainloss = []
trainacc = []
valloss = []
valacc = []
for epoch in range(1, epochs+1):
    total = 0
    correct = 0
    run = 0
    for data in trainloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = net(images).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        run += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    if epoch % step == 0:
        net.eval()
        valcorrect = 0
        valtotal = 0
        vrun = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)
                outputs = net(images).to(device)
                loss = criterion(outputs, labels)
                _, predictions = outputs.max(1)
                valcorrect += (predictions == labels).sum()
                valtotal += predictions.size(0)
                vrun += loss.item()
        valloss.append(vrun)
        valacc.append(100 * valcorrect / valtotal)
        print(f'Validation Loss {valloss[-1]}\tAccuracy {valacc[-1]}')
        net.train()
                
    trainloss.append(run)
    trainacc.append(100 * correct / total)
    print(f'Epoch {epoch}\tLoss {trainloss[-1]}\tAccuracy {trainacc[-1]}')

In [ ]:
def accuracy_of_net(net, loader):
    net.eval()
    num_correct = 0
    num_samples = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images).to(device)
            _, predictions = outputs.max(1)
            num_correct += (predictions == labels).sum()
            num_samples += predictions.size(0)
        accuracy = 100 * num_correct / num_samples
        net.train()
    return accuracy

In [ ]:
train_accuracy = accuracy_of_net(net, trainloader)
validation_accuracy = accuracy_of_net(net, valloader)
test_accuracy = accuracy_of_net(net, testloader)
print(f'Accuracy on training set: {train_accuracy}%')
print(f'Accuracy on validation set: {validation_accuracy}%')
print(f'Accuracy on test set: {test_accuracy}%')

In [ ]:
x = [i for i in range(epochs) if i % step == 0]
tloss = [trainloss[i] for i in x]
tacc = [trainacc[i] for i in x]

In [ ]:
plt.plot(x, tloss)
plt.legend(['training loss'])

In [ ]:
plt.plot(x, tacc)
plt.legend(['training accuracy'])

In [ ]:
plt.plot(x, valloss)
plt.legend(['validation loss'])

In [ ]:
plt.plot(x, valacc)
plt.legend(['validation accuracy'])

In [ ]:
plt.plot(x, tloss, label = 'training loss')
plt.plot(x, valloss, label = 'val loss')
plt.legend(['training loss', 'validation loss'])

In [ ]:
plt.plot(x, tacc, label = 'training accuracy')
plt.plot(x, valacc, label = 'val accuracy')
plt.legend(['training accuracy', 'validation accuracy'], loc = "upper left")

In [ ]:
start = cuda.Event(enable_timing = True)
end = cuda.Event(enable_timing = True)

start.record()
acc = accuracy_of_net(net, trainloader)
acc = accuracy_of_net(net, valloader)
acc = accuracy_of_net(net, testloader)
end.record()

cuda.synchronize()
time = start.elapsed_time(end)
print('Time taken to predict complete dataset = ', time)
print(f'Average time taken per sample = {time / size}')

In [ ]:
def list_to_tensor(x, device):
    if device == 'cuda':
        return torch.cuda.FloatTensor(x)
    return torch.FloatTensor(x)

In [ ]:
def get_metrics(net, loader):
    net.eval()
    pred = []
    target = []
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images).to(device)
        _, predictions = outputs.max(1)
        for p in predictions:
            pred.append(p.item())
        for l in labels:
            target.append(l.item())
    
    pred = list_to_tensor(pred, device)
    target = list_to_tensor(target, device)

    mape = MAPE(target, pred)
    mae = mean_absolute_error(target, pred)
    mse = mean_squared_error(target, pred)
    rmse = root_mean_squared_error(target, pred)
    net.train()
    return {'mape' : mape, 'mae' : mae, 'mse' : mse, 'rmse' : rmse}

In [ ]:
def MAPE(target, prediction):
    n = len(target)
    target = [target[i] + 1 for i in range(n)]
    prediction = [prediction[i] + 1 for i in range(n)]
    s = 0
    for i in range(n):
        s += (abs((target[i] - prediction[i]) / target[i]))
    s /= n
    return s * 100

In [ ]:
def print_metrics(metrics):
    mape = metrics['mape']
    mae = metrics['mae']
    mse = metrics['mse']
    rmse = metrics['rmse']
    print(f'Mean Absolute Percentage Error:\t{mape}')
    print(f'Mean Absolute Error:\t{mae}')
    print(f'Mean Squared Error:\t{mse}')
    print(f'Root Mean Squared Error:\t{rmse}')

In [ ]:
test_metrics = get_metrics(net, testloader)
print_metrics(test_metrics)

In [ ]:
train_metrics = get_metrics(net, trainloader)
print_metrics(train_metrics)

In [ ]:
val_metrics = get_metrics(net, valloader)
print_metrics(val_metrics)

In [ ]:
layers = 0
conv_layers=[]
children = list(net.children())

for child in children:
    if type(child) == Conv2d:
        layers += 1
        conv_layers.append(child)
    elif type(child) == Sequential:
        for layer in child.children():
            if type(layer) == Conv2d:
                layers += 1
                conv_layers.append(layer)
print('Number of convolutional layers = ', layers)

In [ ]:
import numpy as np
from PIL import Image
import cv2 as cv
img = cv.imread("/content/drive/MyDrive/buffer64/happy/image1128.png")
cv2_imshow(img)
arr = np.array(img)
img = Image.fromarray(arr)
img = my_transforms(img)
img = img.unsqueeze(0)

In [ ]:
img = img.to(device)
results = [conv_layers[0](img)]
for i in range(1, len(conv_layers)):
    results.append(conv_layers[i](results[-1]))
outputs = results

In [ ]:
# Feature maps
for num_layer in range(len(outputs)):
    plt.figure(figsize=(50, 10))
    layer_viz = outputs[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print("Layer ",num_layer+1)
    for i, filter in enumerate(layer_viz):
        if i == 16: 
            break
        plt.subplot(2, 8, i + 1)
        cpu_filter = filter.cpu()
        plt.imshow(cpu_filter, cmap='gray')
        plt.axis("off")
    plt.show()
    plt.close()

In [ ]:
# Prediction
import numpy as np
from google.colab.patches import cv2_imshow
from PIL import Image
import cv2 as cv
img = cv.imread("/content/image.png")
cv2_imshow(img)
arr = np.array(img)
img = Image.fromarray(arr)
img = transform(img)
img=img.unsqueeze(0)
img = img.cuda()
output = net(img).to(device)
_, prediction = output.max(1)
target_class = dataset.classes
print(target_class[prediction])